## Amazon AgentCore Bedrock コード インタープリター チュートリアル

このチュートリアルでは、AgentCore Bedrock コード インタープリターを使用して以下の作業を行う方法を示します。
1. サンドボックス環境のセットアップ
2. データの読み込みと分析
3. サンドボックス環境でのコードの実行
4. 結果の処理と取得

## 前提条件
- Bedrock AgentCore コード インタープリターへのアクセス権を持つ AWS アカウント
- コード インタープリター リソースの作成と管理に必要な IAM 権限
- 必要な Python パッケージがインストールされていること (boto3 & bedrock-agentcore を含む)
- サンプル データ ファイル (data.csv)
- 分析スクリプト (stats.py)

## あなたの IAM 実行ロールには、以下の IAM ポリシーが付与されている必要があります

```json
{
 "Version": "2012-10-17",
 "Statement": [
  {
   "Effect": "Allow",
   "Action": [
    "agentcore:CreateCodeInterpreter",
    "agentcore:DeleteCodeInterpreter",
    "agentcore:GetCodeInterpreter",
    "agentcore:ListCodeInterpreters",
    "agentcore:StartCodeInterpreter",
    "agentcore:StopCodeInterpreter",
    "agentcore:UploadCode",
    "agentcore:GetCodeOutput",
    "agentcore:GetCodeStatus"
   ],
   "Resource": "*"
  }
 ]
}
```

~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateCodeInterpreter",
                "bedrock-agentcore:StartCodeInterpreterSession",
                "bedrock-agentcore:InvokeCodeInterpreter",
                "bedrock-agentcore:StopCodeInterpreterSession",
                "bedrock-agentcore:DeleteCodeInterpreter",
                "bedrock-agentcore:ListCodeInterpreters",
                "bedrock-agentcore:GetCodeInterpreter"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
        }
    ]
}

日本語訳:
~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "許可",
            "Action": [
                "bedrock-agentcore:CreateCodeInterpreter",
                "bedrock-agentcore:StartCodeInterpreterSession",
                "bedrock-agentcore:InvokeCodeInterpreter",
                "bedrock-agentcore:StopCodeInterpreterSession",
                "bedrock-agentcore:DeleteCodeInterpreter",
                "bedrock-agentcore:ListCodeInterpreters",
                "bedrock-agentcore:GetCodeInterpreter"
            ],
            "Resource": "*"
        },
        {
            "Effect": "許可",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
        }
    ]
}

## 動作の仕組み

コード実行サンドボックスは、コード インタプリター、シェル、ファイル システムを備えた分離された環境を作成することで、エージェントがユーザーのクエリを安全に処理できるようにします。大規模言語モデルがツールの選択を支援した後、コードがこのセッション内で実行され、その後ユーザーまたはエージェントに合成のために返されます。

![architecture local](code-interpreter.png)

## 1. 環境のセットアップ

まず、必要なライブラリをインポートし、 Code Interpreter クライアントを初期化しましょう。

In [ ]:
!pip install --upgrade -r requirements.txt

In [21]:
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
import json
from typing import Dict, Any, List

# AWS リージョンでコード インタープリターを初期化する

日本語訳:

# AWS リージョンで Code Interpreter を初期化してください。
code_client = CodeInterpreter('us-west-2')
code_client.start()

'01K00Z3F8WZ9KBBW4QGRJCVBHH'

## 2. ローカルファイルの読み込み

ここでは、サンプルデータファイルとanalysis スクリプトの内容を読み込みます。

In [22]:
def read_file(file_path: str) -> str:
    """ファイルの内容を読み込むためのヘルパー関数 (エラー処理付き)"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return ""
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

# Read both files

with open('file1.txt', 'r') as file1, open('file2.txt', 'r') as file2:
    contents1 = file1.read()
    contents2 = file2.read()

print('Contents of file1.txt: ')
print(contents1)

print('Contents of file2.txt: ')
print(contents2)

日本語訳:

# 両方のファイルを読み込む

with open('file1.txt', 'r') as file1, open('file2.txt', 'r') as file2:
    contents1 = file1.read()
    contents2 = file2.read()

print('file1.txt の内容: ')
print(contents1)

print('file2.txt の内容: ')
print(contents2)
data_file_content = read_file("samples/data.csv")
code_file_content = read_file("samples/stats.py")

## 3. Sandbox 環境のファイルの準備

Sandbox 環境で作成したいファイルを定義する構造を作成します。

In [23]:
files_to_create = [
    {
        "path": "data.csv",
        "text": data_file_content
    },
    {
        "path": "stats.py",
        "text": code_file_content
    }
]

以下が日本語訳になります。

## 4. ツール呼び出し用のヘルパー関数の作成

このヘルパー関数を使うと、サンドボックスツールの呼び出しとその応答の処理が簡単になります。アクティブなセッション内では、サポートされている言語 (Python、JavaScript) でコードを実行したり、依存関係の設定に基づいてライブラリにアクセスしたり、可視化を生成したり、実行間で状態を維持したりできます。

In [24]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    以下が日本語訳になります。

"""サンドボックスツールを呼び出すためのヘルパー関数

    引数:
        tool_name (str): 呼び出すツールの名前
        arguments (Dict[str, Any]): ツールに渡す引数

    返り値:
        Dict[str, Any]: JSON 形式の結果
"""
    response = code_client.invoke(tool_name, arguments)
    for event in response["stream"]:
        return json.dumps(event["result"])

## 5. サンドボックスへのファイル書き込み

次に、ファイルをサンドボックス環境に書き込み、正常に作成されたことを確認します。

```python
with open('hello.txt', 'w') as f:
    f.write('Hello World!')

with open('example.py', 'w') as f:
    f.write('print("Hello from Python!")')
```

上記の `open()` 関数を使って、 `hello.txt` と `example.py` という 2 つのファイルを作成しています。最初の引数がファイル名、2 番目の引数 `'w'` が書き込みモードを表します。

`with` 文を使うと、ファイルを自動的に閉じることができるので、リソースの解放を気にする必要がありません。

ファイルが正しく作成されたかを確認するには、サンドボックスの左側にあるファイルツリーを確認します。 `hello.txt` と `example.py` が表示されていれば成功です。

In [25]:
# Write files to sandbox

日本語訳:
# サンドボックスにファイルを書き込む

The `write_file` function allows you to write data to a file in the sandbox. Here's an example:

`write_file` 関数を使うと、サンドボックス内のファイルにデータを書き込むことができます。例を示します:

```python
open('hello.txt', 'w').write('Hello, World!')
```

This will create a new file called `hello.txt` in the sandbox, and write the string `'Hello, World!'` to it.

これにより、サンドボックス内に `hello.txt` という新しいファイルが作成され、その中に文字列 `'Hello, World!'` が書き込まれます。

Note that every time you re-run your code, the sandbox will be reset and any files you've written will be deleted. If you want to keep the files, you'll need to download them before re-running.

コードを再実行するたびにサンドボックスがリセットされ、書き込んだファイルは削除されることに注意してください。ファイルを保持したい場合は、再実行する前にダウンロードする必要があります。
writing_files = call_tool("writeFiles", {"content": files_to_create})
print("Writing files result:")
print(writing_files)

# Verify files were created

ファイルが作成されたことを確認する
listing_files = call_tool("listFiles", {"path": ""})
print("\nFiles in sandbox:")
print(listing_files)

Writing files result:
{"content": [{"type": "text", "text": "Successfully wrote all 2 files"}], "isError": false}

Files in sandbox:
{"content": [{"type": "resource_link", "uri": "file:///log", "name": "log", "description": "Directory"}, {"type": "resource_link", "mimeType": "text/csv", "uri": "file:///data.csv", "name": "data.csv", "description": "File"}, {"type": "resource_link", "mimeType": "text/x-python", "uri": "file:///stats.py", "name": "stats.py", "description": "File"}, {"type": "resource_link", "uri": "file:///.ipython", "name": ".ipython", "description": "Directory"}], "isError": false}


## 6. 分析の実行

ここでは、サンドボックス環境で分析スクリプトを実行し、結果を処理します。

In [26]:
import pprint

# 分析スクリプトを実行する

日本語訳:

分析スクリプトを実行するには、次のコマンドを実行します。

```
python3 analyze_data.py --input_file data.csv --output_dir analysis_output
```

このコマンドでは、 `analyze_data.py` スクリプトが実行され、 `data.csv` ファイルからデータを読み込み、分析結果を `analysis_output` ディレクトリに出力します。

スクリプトの主な機能は以下の通りです。

1. `load_data()` 関数で `data.csv` からデータを読み込む
2. `preprocess_data()` 関数でデータを前処理する
3. `run_analysis()` 関数で実際の分析を実行する
4. `save_results()` 関数で分析結果を `analysis_output` ディレクトリに保存する

分析の詳細な手順は、スクリプトのソースコードを参照してください。
code_execute_result = call_tool("executeCode", {
    "code": files_to_create[1]['text'],
    "language": "python",
    "clearContext": True
})

# Parse and display results

# 結果を解析して表示する

results = parse_data( raw_data )
for result in results:
    print( f"Name: { result['name'] }" )
    print( f"Score: { result['score'] }" )
    print( )  # 空行を挿入

# 結果を JSON ファイルに書き出す
output_json( results, "output.json" )
analysis_results = json.loads(code_execute_result)
print("Full analysis results:")
pprint.pprint(analysis_results)

print("\nStandard output from analysis:")
print(analysis_results['structuredContent']['stdout'])

Full analysis results:
{'content': [{'text': 'Name   Place  Animal   Thing\n'
                      'count       299130  299130  299130  299130\n'
                      'unique        1722      55      50      51\n'
                      'top     Lisa White  Prague    Goat  Pencil\n'
                      'freq           222    5587    6141    6058',
              'type': 'text'}],
 'isError': False,
 'structuredContent': {'executionTime': 0.7111423015594482,
                       'exitCode': 0,
                       'stderr': '',
                       'stdout': 'Name   Place  Animal   Thing\n'
                                 'count       299130  299130  299130  299130\n'
                                 'unique        1722      55      50      51\n'
                                 'top     Lisa White  Prague    Goat  Pencil\n'
                                 'freq           222    5587    6141    6058'}}

Standard output from analysis:
Name   Place  Animal   Thing
count       29

## 7. クリーンアップ

最後に、Code Interpreter セッションを停止してクリーンアップを行います。セッションの使用が終わったら、リソースを解放し、不要な課金を避けるためにセッションを停止する必要があります。

In [27]:
# コードインタープリターセッションを停止する

 session = PromptSession(cli=cli)
 try:
     session.app()
 except KeyboardInterrupt:
     print('無事終了しました')
 finally:
     session.exit()
code_client.stop()
print("Code Interpreter session stopped successfully!")

Code Interpreter session stopped successfully!


## 結論

このチュートリアルでは、以下の内容を学びました:

- Code Interpreter セッションを初期化する方法
- ファイルを読み込み、解析の準備をする方法
- サンドボックス環境でコードを実行する方法
- 結果を処理して表示する方法
- リソースをクリーンアップする方法